In [ ]:
#Ativar o autocomplete
%config IPCompleter.greedy=True

In [ ]:
import json
import pandas as pd
from functools import reduce
import os
from IPython.display import display, HTML

In [ ]:
def display_df(df):
    display(HTML(df.to_html()))

In [ ]:
def json_extract(obj, key):
    """Recursively fetch values from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    values = extract(obj, arr, key)
    return values

In [ ]:
def list_append(obj, key, list):
    extracted_data = json_extract(obj, key)
    if not extracted_data:
        list.append(None)
    else:
        list.append(extracted_data[0])

In [ ]:
def blocks_data_extract(obj):
    branches = []
    dataviewer_getValue_lists = []
    blocks_names = []
    if 'project' in obj:
        lists = {}
        for stage_actors in obj['project']['targets']:
            if stage_actors['lists']:
                for listID, details in stage_actors['lists'].items():
                    lists[listID] = details[0]

        speed = {4: 'high', 3: 'medium', 2: 'low', 1: 'zero'}
        blocks = []
        for stage_actors in obj['project']['targets']:
            if stage_actors['blocks']:
                for blockID, details in stage_actors['blocks'].items():
                    if isinstance(details, dict) and details['shadow'] == False:
                        blockName = details['opcode']
                        blocks_names.append(blockName)
                        if 'LIST_ID' in details['fields']:
                            listID = details['fields']['LIST_ID'][0]
                            if listID in lists:
                                if blockName == 'dataviewer_getValue' and lists[listID] not in dataviewer_getValue_lists:
                                    dataviewer_getValue_lists.append(lists[listID])
                                blockName = blockName + '(' + lists[listID] + ')'
                            else:
                                blockName = blockName + '(' + listID + ')'
                        if 'WHENTEMPMENU' in details['fields']:
                            blockName = blockName + '(' + details['fields']['WHENTEMPMENU'][0] + ')'
                        if 'PARTICLESPEED' in details['fields']:
                            blockName = blockName + '(' + speed[int(details['fields']['PARTICLESPEED'][0])] + ')'
                        blocks.append((blockID, blockName, details['parent']))
        
        branchesID = []
        while len(blocks) > 0:
            i = 0
            while i < len(blocks):
                if blocks[i][2] == None:
                    branches.append([blocks[i][1]])
                    branchesID.append([blocks[i][0]])
                    del blocks[i]
                    i -= 1
                else:
                    for j in range(0, len(branches)):
                        if blocks[i][2] in branchesID[j]:
                            branches[j].append(blocks[i][1])
                            branchesID[j].append(blocks[i][0])
                            del blocks[i]
                            i -= 1
                            break
                i += 1
    return blocks_names, branches, dataviewer_getValue_lists

In [ ]:
projectID = []
time = []
classname = []
event = []
block_name = []
qty_deleted = []
qty_blocks = []
all_blocks_names = []
all_branches = []
qty_branches = []
all_dataviewer_getValue_lists = []
qty_dataviewer_getValue_lists = []
bars_size = 5

In [ ]:
path = 'C:\\Users\\Lucas\\Documents\\ProjetoCassia\\Dados'
folders = [name for name in os.listdir(path) if os.path.isdir(os.path.join(path, name))]
for folder in folders:
    file_path = path + '\\' + folder + '\\timeline.json'
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    time_zero = int(next(iter(data))[0:10])
    for k in data:
        projectID.append(folder)
        time.append(int(k[0:10]) - time_zero)    
        list_append(data[k], 'classname', classname)
        list_append(data[k], 'event', event)
        list_append(data[k], 'block', block_name)
        list_append(data[k], 'deletedBlocks', qty_deleted)
        blocks_data = blocks_data_extract(data[k])
        blocksNames = blocks_data[0]
        if not blocksNames:
            all_blocks_names.append(None)
            qty_blocks.append(None)
        else:
            all_blocks_names.append(blocksNames)
            qty_blocks.append(len(blocksNames))
        branches = blocks_data[1]
        if not branches:
            all_branches.append(None)
            qty_branches.append(None)
        else:
            all_branches.append(branches)
            qty_branches.append(len(branches))
        dataviewer_getValue_lists = blocks_data[2]
        if not dataviewer_getValue_lists:
            all_dataviewer_getValue_lists.append(None)
            qty_dataviewer_getValue_lists.append(None)
        else:
            all_dataviewer_getValue_lists.append(dataviewer_getValue_lists)
            qty_dataviewer_getValue_lists.append(len(dataviewer_getValue_lists))

In [ ]:
df = pd.DataFrame(list(zip(
    projectID, time, classname, event, block_name, qty_deleted,
    qty_blocks, all_blocks_names, qty_branches, all_branches,
    qty_dataviewer_getValue_lists, all_dataviewer_getValue_lists)),
                      columns = ['projectID', 'time', 'classname', 'event', 'block_name', 'qty_delete',
                                 'qty_blocks', 'all_blocks_names', 'qty_branches', 'all_branches',
                                 'qty_dataviewer_getValue_lists', 'all_dataviewer_getValue_lists'])

In [ ]:
display_df(df)

In [ ]:
#Visualização do estado final de cada projeto
display_df(df.loc[df[(df.event == 'PROJECT_CHANGED')].groupby('projectID').time.idxmax()])

In [ ]:
#Importante: a ordem da lista (event_list) não é a mesma ordem que os dados são gerados no analysis_count.
event_list = ['PROJECT_RUN_START', 'PROJECT_RUN_STOP', 'PROJECT_START', 'SCRIPT_GLOW_ON', 'SCRIPT_GLOW_OFF', 'greenFlag*']
analysis_count = df[['projectID', 'event']][df.event.isin(event_list)].groupby(['projectID', 'event']).size().unstack(
    fill_value=0)
analysis_qty_delete = df[['projectID', 'qty_delete']].groupby(['projectID']).agg(['sum'])
analysis_qty_delete.columns = ['Blocos_Apagados']
analysis_qty_blocks_end = df.loc[df[(df.event == 'PROJECT_CHANGED')].groupby('projectID').time.idxmax()][['projectID',
                                                                                                          'qty_blocks']]
analysis_qty_blocks_end.set_index('projectID', inplace=True)
analysis_qty_blocks_end.columns = ['Qtde_Blocos_Fim']
analysis_total_time = df[['projectID', 'time']].groupby(['projectID']).agg(['max'])
analysis_total_time.columns = ['Tempo_Total']
analysis_total_time['Tempo_Total (min)'] = analysis_total_time['Tempo_Total'].div(60).round(0)
analysis_dfs = [analysis_count, analysis_qty_delete, analysis_qty_blocks_end, analysis_total_time]
analysis = reduce(lambda  left, right: pd.merge(left, right, left_index=True, right_index=True, how='outer'), analysis_dfs)
#Ordenando as colunas
analysis = analysis[['PROJECT_RUN_START', 'PROJECT_RUN_STOP', 'SCRIPT_GLOW_ON', 'SCRIPT_GLOW_OFF', 'PROJECT_START',
                     'greenFlag*', 'Blocos_Apagados', 'Qtde_Blocos_Fim', 'Tempo_Total', 'Tempo_Total (min)']]

In [ ]:
#Forma alternativa de montar as análises
#analysis_qty_run_start = df[['projectID', 'event']][(df.event == 'PROJECT_RUN_START')].groupby(['projectID']).agg(['count'])
#analysis_qty_run_stop = df[['projectID', 'event']][(df.event == 'PROJECT_RUN_STOP')].groupby(['projectID']).agg(['count'])
#analysis_qty_glow_on = df[['projectID', 'event']][(df.event == 'SCRIPT_GLOW_ON')].groupby(['projectID']).agg(['count'])
#analysis_qty_glow_off = df[['projectID', 'event']][(df.event == 'SCRIPT_GLOW_OFF')].groupby(['projectID']).agg(['count'])
#analysis_qty_start = df[['projectID', 'event']][(df.event == 'PROJECT_START')].groupby(['projectID']).agg(['count'])
#analysis_qty_greenflag = df[['projectID', 'event']][(df.event == 'greenFlag*')].groupby(['projectID']).agg(['count'])
#analysis_qty_delete = df[['projectID', 'qty_delete']].groupby(['projectID']).agg(['sum'])
#analysis_total_time = df[['projectID', 'time']].groupby(['projectID']).agg(['max'])
#analysis_qty_blocks_end = df.loc[df[(df.event == 'PROJECT_CHANGED')].groupby('projectID').time.idxmax()][['projectID',
#                                                                                                          'qty_blocks']]
#analysis_qty_blocks_end.set_index('projectID', inplace=True)
#analysis_dfs = [analysis_qty_run_start,  analysis_qty_run_stop, analysis_qty_glow_on, analysis_qty_glow_off,
#                analysis_qty_start, analysis_qty_greenflag, analysis_qty_delete, analysis_total_time, analysis_qty_blocks_end]
#analysis = reduce(lambda  left, right: pd.merge(left, right, left_index=True, right_index=True, how='outer'), analysis_dfs)
#analysis.columns = ['RUN_START', 'RUN_STOP', 'GLOW_ON', 'GLOW_OFF',
#                    'START', 'GreenFlag', 'Blocos_Apagados', 'Tempo_Total', 'Qtde_Blocos_Fim']
#analysis['Tempo_Total (min)'] = analysis['Tempo_Total'].div(60).round(0)

In [ ]:
display_df(analysis)

In [ ]:
from pathlib import Path
data_folder = Path(r"C:\Users\Lucas\Documents\ProjetoCassia\Dados\1")
df1 = df[df.projectID == '1']
df_time_event = df1[['time', 'event']]
df_time_classname = df1[['time', 'classname']]
df_blocks = df1[['time', 'qty_blocks']].dropna()

In [ ]:
df_runstop = df_time_event[(df.event == 'PROJECT_RUN_START') | (df.event == 'PROJECT_RUN_STOP')].replace(
    ['PROJECT_RUN_START', 'PROJECT_RUN_STOP'], [bars_size, 0])

In [ ]:
df_glow = df_time_event[(df.event == 'SCRIPT_GLOW_ON') | (df.event == 'SCRIPT_GLOW_OFF')].replace(
    ['SCRIPT_GLOW_ON', 'SCRIPT_GLOW_OFF'], [bars_size, 0])

In [ ]:
df_events_blockdrag = df_time_event[(df.event == 'BLOCK_DRAG_UPDATE')].replace('BLOCK_DRAG_UPDATE', bars_size)

In [ ]:
df_events_backdrop = df_time_classname.replace('SVGSkin', bars_size)
df_events_backdrop.loc[~df['classname'].isin(['SVGSkin']), 'classname'] = 0

In [ ]:
df_events_projectchanged = df_time_event.replace('PROJECT_CHANGED', bars_size)
df_events_projectchanged.loc[~df['event'].isin(['PROJECT_CHANGED']), 'event'] = 0

In [ ]:
df_events_newsprite = df_time_event.replace('addSprite*', bars_size)
df_events_newsprite.loc[~df['event'].isin(['addSprite*']), 'event'] = 0

In [ ]:
df_events_deleteblock = df_time_event.replace('qty_delete*', bars_size)
df_events_deleteblock.loc[~df['event'].isin(['qty_delete*']), 'event'] = 0

In [ ]:
df_events_addblock = df_time_event.replace('createBlock*', bars_size)
df_events_addblock.loc[~df['event'].isin(['createBlock*']), 'event'] = 0

In [ ]:
df_events_deletesprite = df_time_event.replace(['deleteSprite*', 'targetWasRemoved*'], bars_size)
df_events_deletesprite.loc[~df['event'].isin(['deleteSprite*', 'targetWasRemoved*']), 'event'] = 0

In [ ]:
df_events_table = df_time_event[(df.event == 'openDataviewerTable*') | (df.event == 'closeDataviewerTable*')].replace(
    ['openDataviewerTable*', 'closeDataviewerTable*'], [bars_size, 0])

In [ ]:
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
fig = go.Figure()
fig = make_subplots(rows=5, cols=1, 
                    shared_xaxes=True,
                    vertical_spacing=0.1)

fig.add_trace(go.Bar(
    x=df_runstop.time,
    y=df_runstop.event,
    width=5, 
    name = 'Project Running'
), 
    row=2, col=1
)

fig.add_trace(go.Scatter(
    x=df_glow.time,
    y=df_glow.event,
    mode = 'lines',
    name = 'Project Glow',
    fill="tozeroy",
    line = dict(width=1, shape = 'hv') #shape hv define que é uma linha do tipo step
), 
    row=2, col=1
)

fig.add_trace(go.Scatter(
    x=df_events_table.time,
    y=df_events_table.event,
    mode = 'lines',
    name = 'Table open',
    fill="tozeroy",
    line = dict(color='seagreen', width=1, shape = 'hv') 
), 
    row=1, col=1
)

fig.add_trace(go.Scatter(
    x=df_blocks.time,
    y=df_blocks.qty_blocks,
    #mode = 'lines+markers',
    name = 'Blocks used',
    fill="tozeroy",
    line = dict(color='lightsteelblue', width=1, shape = 'hv') 
    ), 
    row=4, col=1
)

fig.add_trace(go.Bar(
    x=df_events_blockdrag.time,
    y=df_events_blockdrag.event, 
    width= 5 , 
    name = 'Block drag'
), 
    row=5, col=1
)

fig.add_trace(go.Bar(
    x=df_events_newsprite.time,
    y=df_events_newsprite.event, 
    width= 5 , #largura da barra - vai variar dependendo do tempo total do arquivo
    name = 'New sprite added'
), 
    row=1, col=1
)

fig.add_trace(go.Bar(
    x=df_events_deletesprite.time,
    y=df_events_deletesprite.event, 
    width= 5 ,
    name = 'Sprite deleted'
), 
    row=1, col=1
)

fig.add_trace(go.Bar(
    x=df_events_deleteblock.time,
    y=df_events_deleteblock.event, 
    width= 5 , 
    name = 'Block deleted'
), 
    row=3, col=1
)

fig.add_trace(go.Bar(
    x=df_events_addblock.time,
    y=df_events_addblock.event, 
    width= 5 , 
    name = 'Block added'
), 
    row=3, col=1
)

fig.add_trace(go.Scatter(
    x = df_events_backdrop.time,
    y = df_events_backdrop.classname,
    name = 'Editing backdrop or costume',
    mode = 'lines',
    fill="tozeroy",
    line = dict(color='midnightblue', width=1, shape = 'hv'),
), 
    row=1, col=1
)

fig.update_layout(
    showlegend=True, 
    plot_bgcolor='rgba(0,0,0,0)',
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=-0.4,
    xanchor="center",
    x=0.5,
    ),
    font = dict(size = 8), 
)

In [ ]:
import glob
import re
import base64

In [ ]:
#Adicionando as imagens (screenshots) ao gráfico
list_images =  []
list_new_images = []

file_images = glob.glob(os.path.join(data_folder, '*.jpg')) #adiciona todas as imagens da pasta a uma lista

#print (file_images)
for fname in file_images:
    res = re.findall("timeline_(\d+).jpg", fname)
    if not res: continue
    file_new_images = res[0] #res[0] é o número da cada arquivo de imagem, correspondente ao tempo em que foi salvo
    list_images.append(res[0]) 
    new_image_time = int(file_new_images[0:10]) - time_zero
    list_new_images.append(new_image_time) 
#print (list_images)

#adding local images to the plot
#adicionando múltiplas imagens - uma para cada run
i=0
length = len(list_images)
for i in range(length):
    image_filename = file_images[i]  #nome da imagem
    screenshot = base64.b64encode(open(data_folder / image_filename, 'rb').read())
    fig.add_layout_image(dict(
        source='data:image/jpg;base64,{}'.format(screenshot.decode()),
        x=list_new_images[i], 
        y=1,
        xref="x",
        yref="paper",
        )
    )

fig.update_layout_images(dict(
        sizex= 10,#(time_events[-1])/8, #tamanho da imagem - tempo total/10
        sizey=4, # time_events[-1])/10, 
        xanchor="right",
        yanchor="bottom"
))